In [12]:
import sqlite3 as sql
import pandas as pd

In [13]:
class SQLiteDB:
    
    def __init__(self, db):
        self.db = db
        self.conn = None
        
    def connect(self):
        """Create a database connection to the SQLite database."""
        try:
            self.conn = sql.connect(self.db)
        except sql.Error as e:
            print(e)
            
    def create_table(self, create_table_sql):
        """Create a table from the create_table_sql statement."""
        try:
            c = self.conn.cursor()
            c.execute(create_table_sql)
            print("Table created successfully")
        except sql.Error as e:
            print(e)
    
    def delete_table(self, table_name):
        """Delete a table from the database."""
        try:
            c = self.conn.cursor()
            c.execute(f"DROP TABLE IF EXISTS {table_name}")
            print(f"Table {table_name} deleted successfully")
        except sql.Error as e:
            print(e)
    
    def show_tables(self):
        """Display all tables in the database."""
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        try:
            c = self.conn.cursor()
            c.execute(query)
            tables = c.fetchall()
            print("Tables in the database:")
            for table in tables:
                print(table[0])
        except sql.Error as e:
            print(e)
    
    def close(self):
        """Close the database connection."""
        if self.conn:
            self.conn.close()
            print("Connection Closed")
            

In [14]:
#create DashMed db

dashmed = SQLiteDB('DashMed.db')

test_table = """
CREATE TABLE test (
  PatientId integer PRIMARY KEY,
  FirstName text NOT NULL,
  LastName text NOT NULL,
  Address text NOT NULL,
  Phone text NOT NULL,
  Sex text NOT NULL,
  Birthdate date NOT NULL,
  Age integer NOT NULL,
  RelatedPatient text,
  MedicalHistory text,
  Medication text  
);
"""

dashmed.connect()
dashmed.create_table(test_table)
dashmed.close()

Table created successfully
Connection Closed


In [15]:
#show tables

dashmed.connect()
dashmed.show_tables()
dashmed.close()

Tables in the database:
test
Connection Closed


In [16]:
#delete tables

dashmed.connect()
dashmed.delete_table('test')
dashmed.close()

Table test deleted successfully
Connection Closed


In [17]:
# confirm db is empty

dashmed.connect()
dashmed.show_tables()
dashmed.close()

Tables in the database:
Connection Closed


In [19]:
class Summary():
    """ Summary class to retrieve patient data from an input patient ID. """
    
    def __init__(self, db):
        self.db = db
        self.PatientId = None

    def input_patient_id(self):
        """ Input the patient ID number. """
        self.PatientId = input('Enter a patient ID: ')

    def patient_exists(self):
        """ Check if the patient ID number does exist in the database. """
        if self.PatientId is None:
            return False
        
        query = f'SELECT COUNT(*) FROM patients WHERE PatientId = {self.PatientId}'

        try:
            self.db.connect()
            cursor = self.db.conn.cursor()
            cursor.execute(query, (self.PatientId,))
            result = cursor.fetchone()
            return result[0] > 0
        
        except sql.Error as e:
            print(e)
            return False

        finally:
            self.db.close()

    def getdata(self):
        """ Return the row containing the patient information as a list. """
        if not self.patient_exists():
            print("Patient ID not valid. Please input valid patient ID.")
            return

        query = f'SELECT * FROM patients WHERE PatientId = {self.PatientId}'

        try:
            self.db.connect()
            cursor = self.db.conn.cursor()
            cursor.execute(query)
            patient_data = [n for n in cursor.fetchone()]
            
            return patient_data
        
        except sql.Error as e:
            print(e)

        finally:
            self.db.close()
